In [5]:
# import libraries
import pandas as pd
import numpy as np
import xgboost as xgb
import ipaddress
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pickle

Load data from the csv file

In [6]:
data = pd.read_csv("/content/Portmap.csv", low_memory=False)

# Convert the Flow ID column to integers
le = LabelEncoder()
data['Flow ID'] = le.fit_transform(data['Flow ID'])

# Convert the Source IP and Destination IP columns to integers
data[' Source IP'] = data[' Source IP'].apply(lambda x: int(ipaddress.IPv4Address(x)))

data[' Destination IP'] = data[' Destination IP'].apply(lambda x: int(ipaddress.IPv4Address(x)))

# Convert the timestamp column to a datetime object
data[' Timestamp'] = pd.to_datetime(data[' Timestamp'], format='%Y-%m-%d %H:%M:%S.%f')

data[' Timestamp'] = pd.to_numeric(data[' Timestamp'])


# Convert the SimillarHTTP column to integers
data['SimillarHTTP'] = le.fit_transform(data['SimillarHTTP'])

In [ ]:
with open('portmap.pkl', 'wb') as f:
  pickle.dump(data, f) 

In [8]:
data = data[~data.isin([np.nan, np.inf, -np.inf]).any(1)]
data = data.dropna()

# Create a mapping dictionary
label_mapping = {"BENIGN": 0, "Portmap": 1}

# Apply the mapping to the ' Label' column
data[' Label'] = data[' Label'].map(label_mapping)

data.head(4)

<ipython-input-8-a874f15f9349>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  data = data[~data.isin([np.nan, np.inf, -np.inf]).any(1)]


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,24,187181,3232248574,0,3758096389,0,0,1541236696964447000,114456999,45,...,28337.112288,98168.0,3.0,9529897.25,351582.631269,10001143.0,9048097.0,0,0,0
1,26,187180,3232248573,0,3758096389,0,0,1541236698506537000,114347504,56,...,121314.911865,420255.0,4.0,9493929.75,351541.079539,9978130.0,8820294.0,0,0,0
2,176563,186913,3232248326,54799,2899905122,443,6,1541236698610576000,36435473,6,...,0.000000,62416.0,62416.0,36373056.00,0.000000,36373056.0,36373056.0,0,0,0
3,50762,187102,3232248326,54800,2899904258,443,6,1541236698610579000,36434705,6,...,0.000000,62413.0,62413.0,36372291.00,0.000000,36372291.0,36372291.0,0,0,0


In [9]:
from IPython.display import display

# Assuming you have a DataFrame called 'data'
data_types = data.dtypes.to_frame().rename(columns={0: 'Data Type'})
display(data_types)

,Data Type
Unnamed: 0,int64
Flow ID,int64
Source IP,int64
Source Port,int64
Destination IP,int64
...,...
Idle Max,float64
Idle Min,float64
SimillarHTTP,int64
Inbound,int64


Split the data into features and Label

In [12]:
X = data.drop(data.columns[-1],  axis=1)
# X = data.drop('Label', axis='columns', inplace = True)
y = data[data.columns[-1]]
y.head()

0    0
1    0
2    0
3    0
4    0
Name:  Label, dtype: int64

In [28]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [69]:
# Train an xgboost model on the train set

xgb_model = xgb.XGBClassifier()

xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [70]:
# Extract the xgboost predictions on the test set
xgb_pred = xgb_model.predict(X_test)

In [71]:
xgb_pred

array([1, 1, 1, ..., 1, 1, 1])

In [72]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Calculate accuracy
accuracy = accuracy_score(y_test, xgb_pred)

# Calculate precision
precision = precision_score(y_test, xgb_pred)

# Calculate recall
recall = recall_score(y_test, xgb_pred)

# Calculate F1 score
f1 = f1_score(y_test, xgb_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0
